<h1><div align='center'><font size='6' color='brick'> Linear Regression</font></div></h1>
<br>

## Importing packages

In [1]:
import pandas as pd 
from io import StringIO 
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

## Loading data

This is an old data set of home prices.

In [2]:

housing = pd.read_csv("housing.csv") 
housing.head()

,crim,zn,river,rm,ptratio,medv
0,0.00632,18.0,0,6.575,15.3,24.0
1,0.02731,0.0,0,6.421,17.8,21.6
2,0.02729,0.0,0,7.185,17.8,34.7
3,0.03237,0.0,0,6.998,18.7,33.4
4,0.06905,0.0,0,7.147,18.7,36.2


## EDA


In [3]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     506 non-null    float64
 1   zn       506 non-null    float64
 2   river    506 non-null    int64  
 3   rm       506 non-null    float64
 4   ptratio  506 non-null    float64
 5   medv     506 non-null    float64
dtypes: float64(5), int64(1)
memory usage: 23.8 KB


There are no categorical variables.